In [1]:
## Importing packages
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import imageio.v2 as imageio  # use v2 API to avoid warning
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [2]:
fontsize= 14
ticksize = 14
figsize = (10, 8)
params_fig = {'font.family':'serif',
    "figure.figsize":figsize,
    'figure.dpi': 80,
    'figure.edgecolor': 'k',
    'font.size': fontsize,
    'axes.labelsize': fontsize,
    'axes.titlesize': fontsize,
    'xtick.labelsize': ticksize,
    'ytick.labelsize': ticksize
}
plt.rcParams.update(params_fig)

In [3]:
## Runge function
def runge(x):
    return 1 / (1 + 25 * x**2)

def poly_interpolation_runge(n):
    # Equally spaced interpolation nodes
    x_nodes = np.linspace(-1, 1, n + 1)
    y_nodes = runge(x_nodes)
    # Polynomial interpolation (Vandermonde / exact interpolation)
    coeffs = np.polyfit(x_nodes, y_nodes, deg=n)
    p = np.poly1d(coeffs)
    return x_nodes, y_nodes, p

In [4]:
## Degree of the polynomial
n_poly = 10  # degree-10 polynomial → 11 interpolation points
x_train_np, y_train_np, p = poly_interpolation_runge(n = n_poly)

In [5]:
## Test data
x_test_np = np.linspace(-1, 1, 1000)
y_true_np = runge(x_test_np)
y_test_poly_np = p(x_test_np)

In [6]:
class NN(nn.Module):
    def __init__(self,
                 input_dim = 1,
                 dim_hidden=128,
                 layers=2,
                 hidden_bias=True,
                 hidden_activation=nn.Tanh,
                 seed=123):
        super().__init__()
        self.input_dim = input_dim
        self.dim_hidden = dim_hidden
        self.layers = layers
        self.hidden_bias = hidden_bias
        self.hidden_activation = hidden_activation
        self.seed = seed

        # Set seed if provided
        if self.seed is not None:
            torch.manual_seed(self.seed)
        
        module = []
        
        # First layer
        module.append(nn.Linear(self.input_dim, self.dim_hidden, bias=self.hidden_bias))
        module.append(self.hidden_activation())

        # Additional hidden layers
        for _ in range(self.layers - 1):
            module.append(nn.Linear(self.dim_hidden, self.dim_hidden, bias=self.hidden_bias))
            module.append(self.hidden_activation())

        module.append(nn.Linear(self.dim_hidden, 1))

        self.q = nn.Sequential(*module)

    def forward(self, x):
        return self.q(x)

In [20]:
def count_parameters(model):
    total_params = 0
    for param in model.parameters():
        total_params += param.numel()  # numel() returns the total number of elements (weights/biases) in the parameter
    return total_params


In [7]:
x_train = torch.from_numpy(x_train_np).float().unsqueeze(dim = 1)
y_train = torch.from_numpy(y_train_np).float().unsqueeze(dim = 1)

train_dataset = TensorDataset(x_train, y_train)
batch_size = len(x_train)
# Create dataloader
train_loader = DataLoader(train_dataset, batch_size= batch_size, shuffle = False)

In [8]:
num_epochs = 401
print_freq = 100

In [9]:
model = NN()

In [21]:
count_parameters(model)

16897

In [10]:
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.99)

In [11]:
x_test = torch.from_numpy(x_test_np).float().unsqueeze(dim = 1)

In [12]:
criterion = nn.MSELoss()

In [13]:
# saving the results for the animation
y_test_history = []   # will store y_test for each epoch

In [14]:
for epoch in range(num_epochs):
    for x, y in train_loader:
        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()

    scheduler.step()

    # --- save y_test ---
    model.eval()
    with torch.no_grad():
        y_test_epoch = model(x_test).cpu().clone()
    model.train()

    y_test_history.append(y_test_epoch)

    if epoch % print_freq == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}")


Epoch [1/401], Loss: 0.20817570
Epoch [101/401], Loss: 0.02765143
Epoch [201/401], Loss: 0.00914267
Epoch [301/401], Loss: 0.00050452
Epoch [401/401], Loss: 0.00002780


In [15]:
y_test_history_np = torch.stack(y_test_history).squeeze().numpy()

In [16]:
y_test_history_np[1,:].shape

(1000,)

In [17]:
# Creating the animation

In [18]:
filenames = []
#for i in range(epoch_outputs.shape[0]):
for i in range(num_epochs):
    y_test_DL = y_test_history_np[i,:]
    for t in range(20):
        plt.plot(x_test_np, y_true_np, label="Runge function", color ="k", linestyle="--" , linewidth=2)
        plt.plot(x_test_np, y_test_poly_np, color = "orange", label="Degree-10 polynomial")
        plt.plot(x_test_np, y_test_DL, color = "blue", label = f"DL solution (optimization step = {i})")
        plt.scatter(x_train_np, y_train_np, color="r", zorder=3, label="Training/Interpolation points")
        plt.title("Deep Learning Interpolation", fontsize=18)
        plt.legend(loc = "upper center")
        # create file name and append it to a list
        filename = f'frame_{i:03d}_{t:02d}.png'
        filenames.append(filename)
        plt.savefig(filename)
        plt.close()# build gif
with imageio.get_writer('runge.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)
    
   